# Checking the accuracy and conistancy of stats collected

In order to determine and test the accuracy of my ScrapeStats algorithm, I read through each teams collected seasons, tally the total amount of offensive points scored, then compare my calculated value with statmuse's database stored at https://www.statmuse.com/nfl/ask/most-total-nfl-points-last-8-years. Thus I am directly testing whether I accurately pulled each games data, as flipping the home and away team will introduce unnecessary error. 

### Reading in functions and libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np


#read in returnFullSeasonStats function from UniversalFunctions.ipynb
from ipynb.fs.full.UniversalFunctions import returnFullSeasonStats # type: ignore

#read in returnSpecificRow function from UniversalFunctions.ipynb
from ipynb.fs.full.UniversalFunctions import returnSpecificRow # type: ignore

#read in extractNumberFromString function from UniversalFunctions.ipynb
from ipynb.fs.full.UniversalFunctions import extractNumberFromString # type: ignore

#read in getSpreadandScores function from UniversalFunctions.ipynb
from ipynb.fs.full.UniversalFunctions import getSpreadandScores # type: ignore

In [2]:
def returnFullSeasonStats(teamName, yearArr):
    """
    Goes into stats folder and reads in the both stats and Info csv files
    Concats them into a single data frame and returns them.

    Args:
        teamName (string): the team name matching the name within Stats File
        yearArr (array of ints): Allows to make sure we can pull specific year ranges
    Returns:
        TeamPDStats: Pandas Data-Frame of team game stats from all seasons collected
        teamPDInfo: Pandas Data-Frame of game info from all seasons collected
    """
    #sets up first iteration so i only grab row labels one time
    firstSeasonStats = True
    firstSeasonInfo = True

    #data frame holding things like total yards for every season collected
    teamPDStats = pd.DataFrame()
    #data frame holding things like weather for every game in every season collected
    teamPDInfo = pd.DataFrame()
    #gets all csv files for a team
    dirList = os.listdir(f"Stats/{teamName}/")
    for file in dirList:
        #makes sure we only grab the years we want
        if(int(file[-8:-4]) not in yearArr):
            continue
        #if stats then --> else, it is the game info. 
        if "Stats" in file:
            #if on the first seasons gathered --> read then concat to master stats
            if firstSeasonStats:
                curr = pd.read_csv(f"Stats/{teamName}/{file}")
                #axis 1 because I want it to concat to columns, not more rows
                teamPDStats = pd.concat([teamPDStats, curr],axis = 1)
                firstSeasonStats = False
            elif not firstSeasonStats:
                curr = pd.read_csv(f"Stats/{teamName}/{file}")
                #drop unwanted labels before concecrating 
                curr = curr.drop(columns=['Unnamed: 0', 'Unnamed: 1'])
                teamPDStats = pd.concat([teamPDStats, curr],axis = 1)
        else:
            #if on the first seasons gathered --> read then concat to master stats
            if firstSeasonInfo:
                curr = pd.read_csv(f"Stats/{teamName}/{file}")
                #axis 1 because I want it to concat to columns, not more rows
                teamPDInfo = pd.concat([teamPDInfo, curr],axis = 1)
                #once not the first season
                firstSeasonInfo = False
            elif not firstSeasonInfo:
                curr = pd.read_csv(f"Stats/{teamName}/{file}")
                #drop unwanted labels
                curr = curr.drop(columns=['Unnamed: 0', 'Unnamed: 1'])
                teamPDInfo = pd.concat([teamPDInfo, curr],axis = 1)
    #returns the collected, concected pds. 
    return teamPDStats, teamPDInfo
 


def returnSpecificRow(teamDataFrame, statName):
    """
    Goes into given dataframe and pulls out a requested row from the stat name

    Args:
        teamDataFrame (pandas Dataframe): passed dataframe that contains row to extract.
        statName (string): Name of row to extract 
    Returns:
        statPD: Pandas Data-Frame of row we want to extract
    """
    #mask to pull the spectific row we want, case insensitive
    mask = teamDataFrame["Unnamed: 1"].str.lower() == statName.lower()
    #creates the dataframe and ignores the first two labeling columns
    statPD = pd.DataFrame(teamDataFrame.loc[mask].iloc[:, 2:].values.flatten())
    return statPD


def extractNumberFromString(givenString):
    """
    Extracts a float from a string with a format. Requires this number to have a decimal point

    Args:
        givenString (string): string with number inside
    Returns:
        extractedFloat (float): the extracted number within the string
    """
    extractedFloat = givenString.extract(r"(\d+\.\d*)").astype(float)
    return extractedFloat

def getSpreadandScores(currFullSeasonStats, currFullSeasonInfo, currTeam):
    """
    parses out a dataframes data includin the spread all the scores,
        the difference in game scores ect. 
    Args:
        currFullSeasonStats (pandas Dataframe): dataframe with all the stats from 
            potentially mutliple seasons.
        
        currFullSeasonInfo (pandas Dataframe): dataframe with all the game info from 
            potentially mutliple seasons. IE spread, weather, ect.

        currTeam (str): the name of team currently interested in
    Returns:
        teamBeatSpread (Dataframe): how many points team beat spread by. Postive means won
        scoreDifference (Dataframe): differnce between team score and opponent score across seasons
        vegasSpread (Dataframe): Vegas predicted point spread
        teamScoreFloat (Dataframe): team of interest scores throughout seasons
        oppScoreFloat (Dataframe): opponents scores throughout seasons
    """
    #3 letter abbrivation 
    teamAbbr = currFullSeasonStats["Unnamed: 0"].iloc[0]
    #geting team scores and opp scores
    teamsScore = returnSpecificRow(currFullSeasonStats, f"{teamAbbr} Score")
    opponentScore = returnSpecificRow(currFullSeasonStats, "Opp Score")
    #getting total score
    print(teamsScore)
    teamScoreFloat = teamsScore[0].astype(float) 
    oppScoreFloat = opponentScore[0].astype(float)
    #getting actual spread
    scoreDifference = pd.DataFrame(teamScoreFloat - oppScoreFloat)
    #getting vegas over under
    overUnder = returnSpecificRow(currFullSeasonInfo, "Over/Under")
    #getting vegas spread
    spread = returnSpecificRow(currFullSeasonInfo, "Vegas Line")
    #This takes the pandas array, with info like "45.5 (over)" and takes just the number
    #extract takes a regular exprexsion (regex) "\d" grabs number before decimal place
    #"/.?" grabs the the decimal point and "\d*" grabs the post decimal place. 
    #() tells .extract to only take what is in the parentheses until white space.
    overUnderNumber = extractNumberFromString(overUnder[0].str)
    spreadNumber =  extractNumberFromString(spread[0].str)
    #takes out nan and label rows
    overUnderNumber = overUnderNumber
    vegasSpread = spreadNumber
    # print(vegasSpread)
    #convering spread to be in format as vegas. 
    for i in range(1,len(scoreDifference)):
        if ("nan" not in str(spread[0][i])) and currTeam not in spread[0][i]:
            #if broncos were favored then switch there real spread to reflect they won
            #locate the row i in first column make it negative. 
            vegasSpread.iloc[i, 0] = -1 * vegasSpread.iloc[i, 0]

    teamBeatSpread = scoreDifference[:] - vegasSpread[:]
    
    return teamBeatSpread, scoreDifference, vegasSpread, teamScoreFloat, oppScoreFloat


In [3]:
#directory where Stats was stored
directory = 'Stats/'

yearArr = [2017,2018,2019,2020,2021,2022,2023, 2024]

#array of each directory within stats
TeamNamesArr= os.listdir(directory)

totalPointsScoredArr = []

for team in TeamNamesArr:
    if team == ".DS_Store":
        continue
    print(team)
    currFullSeasonStats, currFullSeasonInfo = returnFullSeasonStats(team, yearArr)
    teamBeatSpread, scoreDifference, vegasSpread, teamScoreFloat, oppScoreFloat = getSpreadandScores(currFullSeasonStats, currFullSeasonInfo, team)
    totalPointsScoredArr.append(tuple([team,np.sum(teamScoreFloat)]))


Broncos
      0
0    24
1    42
2    16
3    16
4    10
..   ..
128  10
129  14
130  17
131  27
132  23

[133 rows x 1 columns]
Commanders
      0
0    17
1    27
2    27
3    20
4    26
..   ..
131  29
132  15
133  37
134  41
135  16

[136 rows x 1 columns]
Dolphins
      0
0    19
1     6
2    20
3    10
4    20
..   ..
129   1
130   1
131   1
132   0
133   4

[134 rows x 1 columns]
Vikings
      0
0    19
1     9
2    17
3    14
4    20
..   ..
133  12
134  27
135  25
136   9
137   9

[138 rows x 1 columns]
Chargers
      0
0    22
1    26
2    10
3    10
4    16
..   ..
131  10
132  30
133  30
134  14
135  10

[136 rows x 1 columns]
Cheifs
      0
0    40
1    28
2    28
3    34
4    19
..   ..
149  26
150  30
151  29
152  27
153  21

[154 rows x 1 columns]
Titans
      0
0    26
1    37
2    27
3    14
4    10
..   ..
134  10
135  37
136  30
137  13
138  23

[139 rows x 1 columns]
Packers
      0
0    17
1    23
2    27
3    35
4    35
..   ..
135  28
136  24
137  26
138  41
139  

ValueError: could not convert string to float: 'False'

In [ ]:
directory = 'Stats/'

yearArr = [2022,2023, 2024]

#array of each directory within stats
TeamNamesArr= os.listdir(directory)

totalPointsScoredArr = []

currFullSeasonStats, currFullSeasonInfo = returnFullSeasonStats("Cheifs", yearArr)
teamBeatSpread, scoreDifference, vegasSpread, teamScoreFloat, oppScoreFloat = getSpreadandScores(currFullSeasonStats, currFullSeasonInfo, team)

     0
0   27
1   26
2   22
3   17
4   26
..  ..
56  24
57  31
58  20
59  20
60  38

[61 rows x 1 columns]


In [4]:
totalPointsScoredArr

[('Broncos', 2768.0),
 ('Commanders', 3162.0),
 ('Dolphins', 753.0),
 ('Vikings', 3053.0),
 ('Chargers', 3170.0),
 ('Cheifs', 3758.0),
 ('Titans', 2902.0)]